<a href="https://colab.research.google.com/github/joshuaalpuerto/ML-guide/blob/main/RepoMap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
!pip install -qqq grep_ast pygments tree_sitter_languages --progress-bar off
!pip install -qqq spacy networkx matplotlib textacy --progress-bar off
!python -m spacy download en_core_web_sm

2023-11-20 08:33:25.458201: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-20 08:33:25.458277: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-20 08:33:25.458339: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-20 08:33:28.971776: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 19.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
#connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Using NetworkX

**Ref**  
- https://baotramduong.medium.com/how-to-convert-any-text-into-a-graph-of-concepts-5afe5d129aba

**Conclusion**
NOT WORKING

In [28]:
from textacy import preprocessing

preproc = preprocessing.make_pipeline(
    preprocessing.remove.html_tags,
    preprocessing.normalize.bullet_points,
    preprocessing.normalize.hyphenated_words,
    preprocessing.normalize.unicode,
    preprocessing.normalize.whitespace,
    preprocessing.remove.accents,
    # preprocessing.remove.punctuation,
    preprocessing.replace.currency_symbols,
    preprocessing.replace.emails,
    preprocessing.replace.emojis,
    preprocessing.replace.hashtags,
    preprocessing.replace.numbers,
    preprocessing.replace.phone_numbers,
    preprocessing.replace.urls,
    preprocessing.replace.user_handles,
)


In [29]:
import pandas as pd


def load_json_as_pd(file_path, subset):
    data = pd.read_json(file_path)
    return data


def read_data():
  data = load_json_as_pd(file_path='...path', subset='contentPlainText')
  data['cleaned'] = data['contentPlainText'].apply(lambda x: preproc(x))
  data = data.dropna()
  data = data.drop_duplicates(subset='contentPlainText')

  return data


messages_df = read_data()

In [30]:
import spacy
import networkx as nx
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.probability import FreqDist

# Load SpaCy
nlp = spacy.load('en_core_web_sm')
nltk.download('punkt')
nltk.download('stopwords')

# Text Preprocessing
def preprocess_text(text):
    tokens = word_tokenize(text)
    tokens = [word.lower() for word in tokens if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]
    return (' ').join(tokens)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [31]:
# Entity Recognition
def extract_entities(text):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

# Concept Extraction
def extract_concepts(text):
    tokens = preprocess_text(text)
    freq_dist = FreqDist(tokens)
    concepts = freq_dist.most_common(10)  # Extract top 10 concepts
    return [concept[0] for concept in concepts]

# Define Dependency Parsing Function
def build_concept_graph_with_dependencies(text):
    doc = nlp(text)
    G = nx.Graph()

    for token in doc:
        G.add_node(token.text)

        if token.dep_ != 'punct':
            G.add_edge(token.text, token.head.text, relation=token.dep_)

    return G

# Define Graph Visualization Function
def visualize_graph(graph):
    pos = nx.spring_layout(graph)
    nx.draw(graph, pos, with_labels=True, font_weight='bold', node_color='skyblue', edge_color='gray', font_size=8)
    plt.show()

In [ ]:
messages_df['concepts'] =  messages_df['contentPlainText'].apply(lambda x: extract_concepts(x))
messages_df.head(100)

# Code map

In [ ]:
# @title Code for Repo map
import colorsys
import os
import random
import sys
from collections import Counter, defaultdict, namedtuple
from pathlib import Path

import networkx as nx
import pkg_resources
from diskcache import Cache
from grep_ast import TreeContext, filename_to_lang
from pygments.lexers import guess_lexer_for_filename
from pygments.token import Token
from pygments.util import ClassNotFound
from tqdm import tqdm
from tree_sitter_languages import get_language, get_parser


Tag = namedtuple("Tag", "rel_fname fname line name kind".split())

def read_text(self, filename):
  try:
      with open(str(filename), "r", encoding=self.encoding) as f:
          return f.read()
  except FileNotFoundError:
      print(f"{filename}: file not found error")
      return
  except IsADirectoryError:
      print(f"{filename}: is a directory")
      return
  except UnicodeError as e:
      print(f"{filename}: {e}")
      print("Use --encoding to set the unicode encoding.")
      return

class RepoMap:
    CACHE_VERSION = 3
    TAGS_CACHE_DIR = f".aider.tags.cache.v{CACHE_VERSION}"

    cache_missing = False

    warned_files = set()

    def __init__(
        self,
        map_tokens=1024,
        root=None,
        # TODO: implement later
        main_model=None, # change this later to actual LLM
        repo_content_prefix=None,
        verbose=False,
    ):
        self.verbose = verbose

        if not root:
            root = os.getcwd()
        self.root = root

        self.load_tags_cache()

        self.max_map_tokens = map_tokens

        # self.tokenizer = main_model.tokenizer
        self.repo_content_prefix = repo_content_prefix

    def get_repo_map(self, chat_files, other_files):
        if self.max_map_tokens <= 0:
            return

        if not other_files:
            return

        files_listing = self.get_ranked_tags_map(chat_files, other_files)
        if not files_listing:
            return

        # num_tokens = self.token_count(files_listing)
        num_tokens=0
        if self.verbose:
            print(f"Repo-map: {num_tokens/1024:.1f} k-tokens")

        if chat_files:
            other = "other "
        else:
            other = ""

        if self.repo_content_prefix:
            repo_content = self.repo_content_prefix.format(other=other)
        else:
            repo_content = ""

        repo_content += files_listing

        return repo_content

    def token_count(self, string):
        #return len(self.tokenizer.encode(string))
        return 0

    def get_rel_fname(self, fname):
        return os.path.relpath(fname, self.root)

    def split_path(self, path):
        path = os.path.relpath(path, self.root)
        return [path + ":"]

    def load_tags_cache(self):
        path = Path(self.root) / self.TAGS_CACHE_DIR
        if not path.exists():
            self.cache_missing = True
        self.TAGS_CACHE = Cache(path)

    def save_tags_cache(self):
        pass

    def get_mtime(self, fname):
        try:
            return os.path.getmtime(fname)
        except FileNotFoundError:
            print(f"File not found error: {fname}")

    def get_tags(self, fname, rel_fname):
        # Check if the file is in the cache and if the modification time has not changed
        file_mtime = self.get_mtime(fname)
        if file_mtime is None:
            return []

        cache_key = fname
        if cache_key in self.TAGS_CACHE and self.TAGS_CACHE[cache_key]["mtime"] == file_mtime:
            return self.TAGS_CACHE[cache_key]["data"]

        # miss!

        data = list(self.get_tags_raw(fname, rel_fname))

        # Update the cache
        self.TAGS_CACHE[cache_key] = {"mtime": file_mtime, "data": data}
        self.save_tags_cache()
        return data

    def get_tags_raw(self, fname, rel_fname):
        lang = filename_to_lang(fname)
        if not lang:
            return

        language = get_language(lang)
        parser = get_parser(lang)

        # Load the tags queries
        scm_fname = pkg_resources.resource_filename(
            __name__, os.path.join("queries", f"tree-sitter-{lang}-tags.scm")
        )
        query_scm = Path(scm_fname)
        if not query_scm.exists():
            return
        query_scm = query_scm.read_text()

        code = read_text(fname)
        if not code:
            return
        tree = parser.parse(bytes(code, "utf-8"))

        # Run the tags queries
        query = language.query(query_scm)
        captures = query.captures(tree.root_node)

        captures = list(captures)

        saw = set()
        for node, tag in captures:
            if tag.startswith("name.definition."):
                kind = "def"
            elif tag.startswith("name.reference."):
                kind = "ref"
            else:
                continue

            saw.add(kind)

            result = Tag(
                rel_fname=rel_fname,
                fname=fname,
                name=node.text.decode("utf-8"),
                kind=kind,
                line=node.start_point[0],
            )

            yield result

        if "ref" in saw:
            return
        if "def" not in saw:
            return

        # We saw defs, without any refs
        # Some tags files only provide defs (cpp, for example)
        # Use pygments to backfill refs

        try:
            lexer = guess_lexer_for_filename(fname, code)
        except ClassNotFound:
            return

        tokens = list(lexer.get_tokens(code))
        tokens = [token[1] for token in tokens if token[0] in Token.Name]

        for token in tokens:
            yield Tag(
                rel_fname=rel_fname,
                fname=fname,
                name=token,
                kind="ref",
                line=-1,
            )

    def get_ranked_tags(self, chat_fnames, other_fnames):
        defines = defaultdict(set)
        references = defaultdict(list)
        definitions = defaultdict(set)

        personalization = dict()

        fnames = set(chat_fnames).union(set(other_fnames))
        chat_rel_fnames = set()

        fnames = sorted(fnames)

        if self.cache_missing:
            fnames = tqdm(fnames)
        self.cache_missing = False

        for fname in fnames:
            if not Path(fname).is_file():
                if fname not in self.warned_files:
                    if Path(fname).exists():
                        print(
                            f"Repo-map can't include {fname}, it is not a normal file"
                        )
                    else:
                        print(f"Repo-map can't include {fname}, it no longer exists")

                self.warned_files.add(fname)
                continue

            # dump(fname)
            rel_fname = os.path.relpath(fname, self.root)

            if fname in chat_fnames:
                personalization[rel_fname] = 1.0
                chat_rel_fnames.add(rel_fname)

            tags = list(self.get_tags(fname, rel_fname))
            if tags is None:
                continue

            for tag in tags:
                if tag.kind == "def":
                    defines[tag.name].add(rel_fname)
                    key = (rel_fname, tag.name)
                    definitions[key].add(tag)

                if tag.kind == "ref":
                    references[tag.name].append(rel_fname)

        ##
        # dump(defines)
        # dump(references)

        if not references:
            references = dict((k, list(v)) for k, v in defines.items())

        idents = set(defines.keys()).intersection(set(references.keys()))

        G = nx.MultiDiGraph()

        for ident in idents:
            definers = defines[ident]
            for referencer, num_refs in Counter(references[ident]).items():
                for definer in definers:
                    # if referencer == definer:
                    #    continue
                    G.add_edge(referencer, definer, weight=num_refs, ident=ident)

        if not references:
            pass

        if personalization:
            pers_args = dict(personalization=personalization, dangling=personalization)
        else:
            pers_args = dict()

        try:
            ranked = nx.pagerank(G, weight="weight", **pers_args)
        except ZeroDivisionError:
            return []

        # distribute the rank from each source node, across all of its out edges
        ranked_definitions = defaultdict(float)
        for src in G.nodes:
            src_rank = ranked[src]
            total_weight = sum(data["weight"] for _src, _dst, data in G.out_edges(src, data=True))
            # dump(src, src_rank, total_weight)
            for _src, dst, data in G.out_edges(src, data=True):
                data["rank"] = src_rank * data["weight"] / total_weight
                ident = data["ident"]
                ranked_definitions[(dst, ident)] += data["rank"]

        ranked_tags = []
        ranked_definitions = sorted(ranked_definitions.items(), reverse=True, key=lambda x: x[1])

        # dump(ranked_definitions)

        for (fname, ident), rank in ranked_definitions:
            # print(f"{rank:.03f} {fname} {ident}")
            if fname in chat_rel_fnames:
                continue
            ranked_tags += list(definitions.get((fname, ident), []))

        rel_other_fnames_without_tags = set(
            os.path.relpath(fname, self.root) for fname in other_fnames
        )

        fnames_already_included = set(rt[0] for rt in ranked_tags)

        top_rank = sorted([(rank, node) for (node, rank) in ranked.items()], reverse=True)
        for rank, fname in top_rank:
            if fname in rel_other_fnames_without_tags:
                rel_other_fnames_without_tags.remove(fname)
            if fname not in fnames_already_included:
                ranked_tags.append((fname,))

        for fname in rel_other_fnames_without_tags:
            ranked_tags.append((fname,))

        return ranked_tags

    def get_ranked_tags_map(self, chat_fnames, other_fnames=None):
        if not other_fnames:
            other_fnames = list()

        ranked_tags = self.get_ranked_tags(chat_fnames, other_fnames)
        num_tags = len(ranked_tags)

        lower_bound = 0
        upper_bound = num_tags
        best_tree = None

        while lower_bound <= upper_bound:
            middle = (lower_bound + upper_bound) // 2
            tree = self.to_tree(ranked_tags[:middle])
            num_tokens = self.token_count(tree)

            if num_tokens < self.max_map_tokens:
                best_tree = tree
                lower_bound = middle + 1
            else:
                upper_bound = middle - 1

        return best_tree

    def to_tree(self, tags):
        if not tags:
            return ""

        tags = sorted(tags)

        cur_fname = None
        context = None
        output = ""

        # add a bogus tag at the end so we trip the this_fname != cur_fname...
        dummy_tag = (None,)
        for tag in tags + [dummy_tag]:
            this_fname = tag[0]

            # ... here ... to output the final real entry in the list
            if this_fname != cur_fname:
                if context:
                    context.add_context()
                    output += "\n"
                    output += cur_fname + ":\n"
                    output += context.format()
                    context = None
                elif cur_fname:
                    output += "\n" + cur_fname + "\n"

                if type(tag) is Tag:
                    code = read_text(tag.fname) or ""

                    context = TreeContext(
                        tag.rel_fname,
                        code,
                        color=False,
                        line_number=False,
                        child_context=False,
                        last_line=False,
                        margin=0,
                        mark_lois=False,
                        loi_pad=0,
                        # header_max=30,
                        show_top_of_file_parent_scope=False,
                    )
                cur_fname = this_fname

            if context:
                context.add_lines_of_interest([tag.line])

        return output


def find_src_files(directory):
    if not os.path.isdir(directory):
        return [directory]

    src_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            src_files.append(os.path.join(root, file))
    return src_files


def get_random_color():
    hue = random.random()
    r, g, b = [int(x * 255) for x in colorsys.hsv_to_rgb(hue, 1, 0.75)]
    res = f"#{r:02x}{g:02x}{b:02x}"
    return res

In [ ]:
!git clone https://github.com/joshuaalpuerto/node-ddd-boilerplate.git

Cloning into 'node-ddd-boilerplate'...
remote: Enumerating objects: 981, done.
remote: Counting objects: 100% (266/266), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 981 (delta 224), reused 210 (delta 196), pack-reused 715
Receiving objects: 100% (981/981), 1.73 MiB | 15.27 MiB/s, done.
Resolving deltas: 100% (505/505), done.


In [ ]:
chat_fnames=find_src_files("src/")

In [ ]:
rm = RepoMap(
    root=".",
    # verbose=True,
)
print(rm.get_ranked_tags_map(chat_fnames))

Repo-map can't include src, it no longer exists

